In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install flask-cors
!pip install pyngrok

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyngrok import ngrok
import os
from dotenv import load_dotenv

load_dotenv()

ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))


In [4]:
import json
import random
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
from unsloth import FastLanguageModel, get_chat_template, is_bfloat16_supported
from unsloth.chat_templates import standardize_sharegpt, train_on_responses_only
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
import torch
from transformers import TextIteratorStreamer
from threading import Thread
from transformers import TrainerCallback

# Add these imports at the top of your file
import zipfile
import io
import shutil
from flask import send_file
# Initialize Flask app
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})  # Enable CORS for all routes

# Global variables
model = None
tokenizer = None
alpaca_prompt = '''You are {name}, a student currently in semester {current_semester} of a {bachelor_specialization} bachelor's at {bachelor_university}, with a CGPA of {current_cgpa}. You have not completed your bachelor's yet. You live at {address}, and your hometown is {hometown}.

  ### Education:
  - High School: {high_school}, studied {_10th_subjects_count} subjects ({_10th_subjects}), scored {_10th_score}.
  - Higher Secondary: {higher_secondary}, studied {_12th_subjects_count} subjects ({_12th_subjects}), scored {_12th_score}.

  ### Skills:
  - Programming: {skills}
  - Tech Stacks: {tech_stacks}

  ### Projects:
  {proj}

  ### Certifications:
  {certi}

  ### Instruction:
  Answer the following question as {name} using only the information provided above. Do not speculate, invent details, or answer questions unrelated to this data. If the question cannot be answered with the given information, politely say, "I don’t have enough information to answer that."
  ### Input:
  {input}
  ### Response:

  '''
user_details = None
dataset_path = "/content/drive/MyDrive/Resources/dataset.json"
model_save_path = "/content/drive/MyDrive/Resources/lora_model"

# Create dataset from user info

training_progress = {"progress": 0, "status": "idle"}
# Generator functions for QA pairs
def generate_name_qa(name):
    questions = ["What’s your name?", "Can you tell me your name?", "Who are you?"]
    answers = [f"My name is {name}.", f"I’m {name}.", f"Sure, my name is {name}."]
    return [(q, a) for q in questions for a in answers]

def generate_address_qa(address):
    questions = [
        "Where do you live?",
        "What is your address?",
        "Where are you located?",
        "What’s your current address?",
        "Where do you reside?"
    ]
    return [(q, f"I live at {address}.") for q in questions]

def generate_hometown_qa(hometown):
    questions = ["Where are you from?", "What’s your hometown?", "Where did you grow up?"]
    return [(q, f"I’m from {hometown}.") for q in questions]

def generate_education_qa(education):
    qa_pairs = [
        (f"How many subjects were there in your 10th grade?", f"There were {education['10th_subjects_count']} subjects in my 10th grade."),
        (f"What subjects did you study in 10th grade?", f"In 10th grade, I studied {', '.join(education['10th_subjects'])}."),
        (f"What was your total score in 10th grade?", f"I scored {education['10th_score']} in 10th grade."),
        (f"How many subjects were there in your 12th grade?", f"There were {education['12th_subjects_count']} subjects in my 12th grade."),
        (f"What subjects did you study in 12th grade?", f"In 12th grade, I studied {', '.join(education['12th_subjects'])}."),
        (f"What was your total score in 12th grade?", f"I scored {education['12th_score']} in 12th grade."),
        (f"Where did you complete your 10th grade?", f"I completed my 10th grade at {education['high_school']}."),
        (f"Where did you complete your 12th grade?", f"I completed my 12th grade at {education['higher_secondary']}.")
    ]
    if education["bachelors_status"] == "yes":
        qa_pairs.append(("What’s your Bachelor’s degree?", f"I have a Bachelor’s in {education['bachelor_specialization']} from {education['bachelor_university']} with a CGPA of {education['final_cgpa']}."))
    else:
        qa_pairs.extend([
            ("Which semester are you in right now?", f"I’m in my {education['current_semester']} semester."),
            ("What’s your current CGPA?", f"My current CGPA is {education['current_cgpa']}.")
        ])
    return qa_pairs

def generate_skills_qa(skills):
    questions = [
        "What are your skills?",
        "What skills do you have?",
        "What are you good at?",
        "Tell me about your abilities."
    ]
    return [(q, f"My skills include {', '.join(skills)}.") for q in questions]

def generate_certifications_qa(certifications):
    questions = ["What certifications do you have?"]
    cert_names = [c["name"] for c in certifications]
    qa_pairs = [(questions[0], f"I have certifications such as {', '.join(cert_names)}.")]
    for c in certifications:
        cert_questions = [
            f"Tell me about your {c['name']} certification.",
            f"What is your {c['name']} certification?",
            f"Can you explain your {c['name']} cert?"
        ]
        qa_pairs.extend([(q, f"I earned the {c['name']} certification from {c['issuer']} in {c['year']}.") for q in cert_questions])
    return qa_pairs

def generate_projects_qa(projects):
    questions = ["What projects have you worked on?"]
    project_names = [p["name"] for p in projects]
    qa_pairs = [(questions[0], f"I have worked on projects such as {', '.join(project_names)}.")]
    for p in projects:
        proj_questions = [
            f"Tell me about {p['name']}.",
            f"What is {p['name']}?",
            f"Can you describe your {p['name']} project?",
            f"What did you do in {p['name']}?"
        ]
        qa_pairs.extend([(q, f"{p['name']} is a project where {p['description']}.") for q in proj_questions])
    return qa_pairs

def generate_tech_stacks_qa(tech_stacks):
    questions = ["What tech stacks are you comfortable with?", "Which technologies do you use?"]
    return [(q, f"I’m comfortable with {', '.join(tech_stacks)}.") for q in questions]

def generate_profession_qa(profession):
    questions = ["What do you do?", "What’s your profession?"]
    return [(q, f"I’m a {profession['title']}, working on {profession['description']}.") for q in questions]

def create_user_details(details): # Store for later use

    # Process education data
    education = {
    "high_school": details.get("Where did you complete your 10th grade? ", ""),
    "10th_subjects_count": details.get("How many subjects in 10th grade? ", ""),
    "10th_subjects": details.get("Subjects in 10th grade (comma-separated): ", "").split(","),
    "10th_score": details.get("Total score in 10th grade: ", ""),
    "higher_secondary": details.get("Where did you complete your 12th grade? ", ""),
    "12th_subjects_count": details.get("How many subjects in 12th grade? ", ""),
    "12th_subjects": details.get("Subjects in 12th grade (comma-separated): ", "").split(","),
    "12th_score": details.get("Total score in 12th grade: ", ""),
    "bachelor_university": details.get("Where are you pursuing your Bachelor's? ", ""),
    "bachelor_specialization": details.get("What's your Bachelor's specialization? ", ""),
    "bachelors_status": details.get("Completed your Bachelor's? (yes/no): ", "no").lower(),
    "current_semester": details.get("What is your current semester? ", ""),
    "current_cgpa": details.get("What is your current CGPA? ", "")
    }

    # Process project data
    num_projects = int(details.get("How many projects do you want to add? ", "0"))
    projects = []
    for i in range(num_projects):
        name_key = f"Enter name of project {i + 1}: "
        desc_key = f"Enter description for project {i + 1}: "
        if name_key in details and desc_key in details:
            projects.append({
                "name": details[name_key],
                "description": details[desc_key]
            })

    # Process certification data
    num_certs = int(details.get("How many certificates do you want to add? ", "0"))
    certifications = []
    for i in range(num_certs):
        name_key = f"Enter name of certificate {i + 1}: "
        issuer_key = f"Enter issuing organization for certificate {i + 1}: "
        year_key = f"Enter year of completion for certificate {i + 1}: "
        if name_key in details and issuer_key in details and year_key in details:
            certifications.append({
                "name": details[name_key],
                "issuer": details[issuer_key],
                "year": details[year_key]
            })

    # Create formatted data structure
    formatted_details = {
        "name": details.get("Enter your name: ", ""),
        "address": details.get("Enter your address: ", ""),
        "hometown": details.get("Enter your hometown: ", ""),
        "education": education,
        "tech_stacks": details.get("Tech stacks you're comfortable with (comma-separated): ", "").split(","),
        "skills": details.get("Enter your Skills (comma-separated): ", "").split(","),
        "projects": projects,
        "certifications": certifications
    }

    # Store the formatted details for chat function
    with open("/content/drive/MyDrive/Resources/user_details.json", "w") as f:
        json.dump(formatted_details, f, indent=4)

    return formatted_details


def create_dataset(details,desired_samples=1000):
    qa_pairs = []
    qa_pairs.extend(generate_name_qa(details["name"]))
    qa_pairs.extend(generate_address_qa(details["address"]))
    qa_pairs.extend(generate_hometown_qa(details["hometown"]))
    qa_pairs.extend(generate_education_qa(details["education"]))
    qa_pairs.extend(generate_tech_stacks_qa(details["tech_stacks"]))
    qa_pairs.extend(generate_certifications_qa(details["certifications"]))
    qa_pairs.extend(generate_skills_qa(details["skills"]))
    qa_pairs.extend(generate_projects_qa(details["projects"]))

    if "profession" in details:
        qa_pairs.extend(generate_profession_qa(details["profession"]))

    # Remove duplicates and limit to desired samples
    unique_qa_pairs = list(set(qa_pairs))
    dataset = []
    for q, a in unique_qa_pairs[:desired_samples]:
        dataset.append({"role": "user", "content": q})
        dataset.append({"role": "assistant", "content": a})

    # Save to JSON
    with open("/content/drive/MyDrive/Resources/dataset.json", "w") as f:
        json.dump(dataset, f, indent=4)
    print(f"Generated {len(dataset)//2} unique QA pairs. Saved to 'dataset.json'.")

    # Show a few examples
    for i in range(0, min(6, len(dataset)), 2):
        print(f"User: {dataset[i]['content']}")
        print(f"Assistant: {dataset[i+1]['content']}")
        print("---")

    return "success"

class ProgressCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        global training_progress
        if state.max_steps > 0:
            progress = (state.global_step / state.max_steps) * 100
            training_progress["progress"] = int(progress)
            training_progress["status"] = "training"
            print(f"Training progress: {int(progress)}%")

    def on_train_end(self, args, state, control, **kwargs):
        global training_progress
        training_progress["progress"] = 100
        training_progress["status"] = "completed"
        print("Training completed")

def train_model():
    global training_progress, model, tokenizer
    model,tokenizer = None,None
    try:
        # Load the pre-trained model and tokenizer with 4-bit quantization
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name="unsloth/Llama-3.2-3B-Instruct",
            max_seq_length=2048,
            dtype=None,
            load_in_4bit=True,
        )

        # Configure LoRA for efficient fine-tuning
        model = FastLanguageModel.get_peft_model(
            model,
            r=16,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
            lora_alpha=16,
            lora_dropout=0,
            bias="none",
            use_gradient_checkpointing="unsloth",
            random_state=3407,
            use_rslora=False,
            loftq_config=None,
        )

        # Apply the appropriate chat template to the tokenizer
        tokenizer = get_chat_template(
            tokenizer,
            chat_template="llama-3.1",
        )

        # Define the function to format prompts with the chat template
        def formatting_prompts_func(examples):
            convos = examples["conversations"]
            texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
            return {"text": texts}

        # Load and prepare the dataset
        dataset_path = "/content/drive/MyDrive/Resources/dataset.json"  # Adjust path as needed
        dataset = load_dataset("json", data_files=dataset_path, split="train")

        def restructure_data(example):
            return {
                "conversations": [
                    {"role": example["role"], "content": example["content"]}
                ]
            }

        dataset = dataset.map(restructure_data, batched=False)
        dataset = standardize_sharegpt(dataset)
        dataset = dataset.map(formatting_prompts_func, batched=True)
        dataset = dataset.remove_columns([col for col in dataset.column_names if col != "text"])

        # Set up the supervised fine-tuning trainer
        trainer = SFTTrainer(
            model=model,
            tokenizer=tokenizer,
            train_dataset=dataset,
            dataset_text_field="text",
            max_seq_length=2048,
            data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
            dataset_num_proc=2,
            packing=False,
            args=TrainingArguments(
                per_device_train_batch_size=2,
                gradient_accumulation_steps=4,
                warmup_steps=5,
                max_steps=60,  # Number of training steps
                learning_rate=2e-4,
                fp16=not is_bfloat16_supported(),
                bf16=is_bfloat16_supported(),
                logging_steps=1,
                optim="adamw_8bit",
                weight_decay=0.01,
                lr_scheduler_type="linear",
                seed=3407,
                output_dir="outputs",
                report_to="none",
            ),
            callbacks=[ProgressCallback()],
        )

        # Modify trainer to train only on assistant responses
        trainer = train_on_responses_only(
            trainer,
            instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
            response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
        )

        # Start the training process
        trainer_stats = trainer.train()

        # Update progress upon completion
        training_progress["progress"] = 100
        training_progress["status"] = "completed"
        print("Training completed, status updated to 'completed'")

        # Save the fine-tuned model and tokenizer
        model_save_path = "/content/drive/MyDrive/Resources/lora_model"
        model.save_pretrained(model_save_path)
        tokenizer.save_pretrained(model_save_path)
        print("Model saved successfully")

    except Exception as e:
        # Handle any errors during training
        training_progress["status"] = "error"
        training_progress["message"] = str(e)
        training_progress["progress"] = 0
        print(f"Training error: {str(e)}")

def load_model():
    """Load the model at startup and keep it in memory."""
    global model, tokenizer, alpaca_prompt, model_save_path
    if model is not None and tokenizer is not None:
        print("Model already loaded at startup.")
        return

    print("Loading model at startup...")
    if not os.path.exists(model_save_path):
        raise FileNotFoundError(f"Model directory {model_save_path} does not exist. Ensure training completed.")

    try:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_save_path,
            max_seq_length=2048,
            dtype=None,
            load_in_4bit=True,
        )
        FastLanguageModel.for_inference(model)
        alpaca_prompt = '''You are {name}, a student currently in semester {current_semester} of a {bachelor_specialization} bachelor's at {bachelor_university}, with a CGPA of {current_cgpa}. You have not completed your bachelor's yet. You live at {address}, and your hometown is {hometown}.

  ### Education:
  - High School: {high_school}, studied {_10th_subjects_count} subjects ({_10th_subjects}), scored {_10th_score}.
  - Higher Secondary: {higher_secondary}, studied {_12th_subjects_count} subjects ({_12th_subjects}), scored {_12th_score}.

  ### Skills:
  - Programming: {skills}
  - Tech Stacks: {tech_stacks}

  ### Projects:
  {proj}

  ### Certifications:
  {certi}

  ### Instruction:
  Answer the following question as {name} using only the information provided above. Do not speculate, invent details, or answer questions unrelated to this data. If the question cannot be answered with the given information, politely say, "I don’t have enough information to answer that."
  ### Input:
  {input}
  ### Response:

  '''
        print("Model loaded successfully at startup!")
    except Exception as e:
        print(f"Failed to load model: {str(e)}")
        raise

def generate_response(query):
    global model, tokenizer, alpaca_prompt, user_details
    if model is None or tokenizer is None:
        load_model()

    # Load user details if not already loaded
    if user_details is None:
        try:
            with open("/content/drive/MyDrive/Resources/user_details.json", "r") as f:
                user_details = json.load(f)
        except Exception as e:
            return f"Error: User details not found. Please complete the fine-tuning process first. Details: {str(e)}"

    # Extract and format personal details
    name = user_details['name'].strip()
    education = user_details['education']
    tech_stacks = ", ".join(user_details['tech_stacks']).replace("AWS,AZURE", "AWS, Azure")
    projects = user_details['projects']
    certifications = user_details['certifications']
    skills = ", ".join(user_details['skills'])
    hometown = user_details['hometown'].strip()
    address = user_details['address'].strip()

    # Format education details with proper capitalization and structure
    high_school = education['high_school'].strip()
    higher_secondary = education['higher_secondary'].strip()
    bachelor_university = education['bachelor_university'].strip()
    current_semester = education.get('current_semester', '3rd').strip()
    current_cgpa = education.get('current_cgpa', '8.5').strip()
    _10th_subjects_count = education['10th_subjects_count'].strip()
    _10th_subjects = ", ".join(education['10th_subjects']).strip()
    _10th_score = education['10th_score'].strip()
    _12th_subjects_count = education['12th_subjects_count'].strip()
    _12th_subjects = ", ".join(education['12th_subjects']).strip()
    _12th_score = education['12th_score'].strip()
    bachelor_specialization = education['bachelor_specialization'].strip()

    # Enhanced projects formatting with more professional descriptions
    proj = ""
    for i, p in enumerate(projects, 1):
        project_name = p['name'].strip()
        project_desc = p['description'].strip()
        # Ensure description ends with period
        if not project_desc.endswith('.'):
            project_desc += '.'
        proj += f"- Project {i}: {project_name} - {project_desc}\n"

    # Enhanced certifications formatting with consistent styling
    certi = ""
    for i, c in enumerate(certifications, 1):
        cert_name = c['name'].strip()
        cert_issuer = c['issuer'].strip()
        cert_year = c['year'].strip()
        certi += f"- Certification {i}: {cert_name} from {cert_issuer} ({cert_year})\n"

    # Populate the prompt with all formatted information
    prompt = alpaca_prompt.format(
        name=name,
        current_semester=current_semester,
        bachelor_university=bachelor_university,
        bachelor_specialization=bachelor_specialization,
        current_cgpa=current_cgpa,
        address=address,
        hometown=hometown,
        high_school=high_school,
        _10th_subjects_count=_10th_subjects_count,
        _10th_subjects=_10th_subjects,
        _10th_score=_10th_score,
        higher_secondary=higher_secondary,
        _12th_subjects_count=_12th_subjects_count,
        _12th_subjects=_12th_subjects,
        _12th_score=_12th_score,
        skills=skills,
        tech_stacks=tech_stacks,
        proj=proj,
        certi=certi,
        input=query
    )
    print(prompt)
    # Enhanced generation parameters for more professional responses
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # Increase max_new_tokens for more comprehensive responses
    # Lower temperature for more focused and professional tone
    generation_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=200,  # Increased for more detailed responses
        do_sample=True,      # Enable sampling for natural language
        top_p=0.92,          # Slightly more focused sampling
        top_k=40,            # More restrictive for professional tone
        temperature=0.7,     # Balanced between creativity and professionalism
        repetition_penalty=1.1  # Avoid repetitive phrases
    )

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # Collect generated text
    generated_text = ""
    for new_text in streamer:
        generated_text += new_text

    # Post-process to ensure professional formatting
    generated_text = generated_text.strip()

    # Ensure response doesn't end mid-sentence
    if generated_text and generated_text[-1] not in ['.', '!', '?', ':']:
        # Find the last complete sentence
        last_period = max(generated_text.rfind('.'), generated_text.rfind('!'), generated_text.rfind('?'))
        if last_period > len(generated_text) * 0.7:  # Only trim if we're not losing too much content
            generated_text = generated_text[:last_period+1]

    return generated_text

# API Routes


@app.route('/ping', methods=['GET'])
def ping():
    """Simple endpoint to verify connectivity"""
    return jsonify({
        "status": "connected",
        "message": "Backend is running",
        "training_status": training_progress
    })

@app.route('/generate_dataset', methods=['POST'])
def api_generate_dataset():
    data = request.get_json()
    if not data:
        print("No data received in request")
        return jsonify({"error": "No data provided"}), 400

    # Log the received data
    print("Received data:", json.dumps(data, indent=4))

    try:
        personal_data = create_user_details(data)
        success = create_dataset(personal_data)
        print("Dataset creation completed successfully")
        return jsonify({
            "success": success,
            "message": "Dataset Creation Successful",
            "status": "completed"
        })
    except Exception as e:
        print(f"Error during dataset creation: {str(e)}")
        return jsonify({"error": str(e), "status": "error"}), 500

@app.route('/train_model', methods=['POST'])
def api_train_model():
    global training_progress
    training_progress = {"progress": 0, "status": "training"}
    try:
        # Run training in a separate thread to avoid blocking
        def run_training():
            try:
                train_model()
            except Exception as e:
                # Make sure exceptions in the thread are captured
                training_progress["status"] = "error"
                training_progress["message"] = str(e)
                print(f"Training error in thread: {str(e)}")

        train_thread = Thread(target=run_training)
        train_thread.daemon = True  # Make thread daemon so it doesn't block app shutdown
        train_thread.start()

        return jsonify({"success": True, "message": "Model training started"})
    except Exception as e:
        training_progress["status"] = "error"
        training_progress["message"] = str(e)
        return jsonify({"error": str(e), "success": False}), 500

@app.route('/load_model', methods=['POST'])
def api_load_model():
    try:
        success = load_model()
        return jsonify({"success": success, "message": "Model loaded successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/chat', methods=['POST'])
def api_chat():
    data = request.get_json()
    if not data or 'message' not in data:
        return jsonify({"error": "No message provided"}), 400

    try:
        response = generate_response(data['message'])
        return jsonify({"response": response})
    except Exception as e:
        return jsonify({"error": str(e)}), 500




@app.route('/download_dataset', methods=['GET'])
def download_dataset():
    """Endpoint to download the generated dataset"""
    try:
        # Check if the dataset file exists
        dataset_file_path = "/content/drive/MyDrive/Resources/dataset.json"
        if not os.path.exists(dataset_file_path):
            return jsonify({"error": "Dataset file not found"}), 404

        # Return the file as an attachment
        return send_file(
            dataset_file_path,
            mimetype='application/json',
            as_attachment=True,
            download_name='dataset.json'
        )
    except Exception as e:
        return jsonify({"error": f"Failed to download dataset: {str(e)}"}), 500

@app.route('/download_model', methods=['GET'])
def download_model():
    """Endpoint to download the trained model as a zip file"""
    try:
        # Check if the model directory exists
        model_dir_path = "/content/drive/MyDrive/Resources/lora_model"
        if not os.path.exists(model_dir_path):
            return jsonify({"error": "Model directory not found"}), 404

        # Create a memory file for the zip
        memory_file = io.BytesIO()

        # Create the zip file
        with zipfile.ZipFile(memory_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Walk through the model directory and add all files
            for root, dirs, files in os.walk(model_dir_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, os.path.dirname(model_dir_path))
                    zipf.write(file_path, arcname)

        # Reset file pointer
        memory_file.seek(0)

        # Return the zip file as an attachment
        return send_file(
            memory_file,
            mimetype='application/zip',
            as_attachment=True,
            download_name='lora_model.zip'
        )
    except Exception as e:
        return jsonify({"error": f"Failed to download model: {str(e)}"}), 500



@app.route('/progress', methods=['GET'])
def api_progress():
    global training_progress
    # Add timestamp to help debug
    response = training_progress.copy()
    response["timestamp"] = time.time()
    return jsonify(response)

if __name__ == '__main__':
    # Start Flask in a separate thread
    def run_flask():
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

    flask_thread = Thread(target=run_flask)
    flask_thread.start()
    print("Flask server started on http://0.0.0.0:5000")

    # Set up ngrok tunnel
    public_url = ngrok.connect(5000, "http")
    print(f"Public URL: {public_url}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Flask server started on http://0.0.0.0:5000
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Public URL: NgrokTunnel: "https://35d6-34-125-38-149.ngrok-free.app" -> "http://localhost:5000"
